In [17]:
import scanpy as sc
import numpy as np
import csv
import pickle
import matplotlib
import math
import json
from itertools import combinations
import pandas as pd
import matplotlib
from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
from cdlib import algorithms
from cdlib import NodeClustering
import altair as alt
import sys
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

spot_diameter = 89.43 #pixels

In [23]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_experiments/exp1_C1_gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_experiments/exp1_C1_coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_experiments/exp1_C1_cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)


# Load pathologist's label
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]


In [24]:
# Make barcode_info
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
barcode_info.append(["cell_code", "x-coordinate", "y-coordinate", "component", "color", "barcode_number"]) # add title
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0, 0, 0, 0]) # last entries will hold the component number later
    i=i+1


barcode_coordinate_dict = {item[0]: [item[1], item[2]] for item in barcode_info}

combined_barcode = {}
for key in barcode_type:
    if key in barcode_coordinate_dict:
        combined_barcode[key] = [barcode_type[key]] + barcode_coordinate_dict[key]


combined_barcode_list = []

for key, values in combined_barcode.items():
    temp_dict = {'barcode': key, 'type': values[0], 'x': values[1], 'y': values[2]}
    combined_barcode_list.append(temp_dict)

In [25]:
# calculate euclidean distance between two points
def euclidean_distance(p1, p2):
    return math.sqrt((p1['x'] - p2['x'])**2 + (p1['y'] - p2['y'])**2)

max_distance = 0
max_distance_barcodes = ()

# Iterate through all pairs of dictionaries and calculate distances - Max
for i, dict1 in enumerate(combined_barcode_list):
    for j, dict2 in enumerate(combined_barcode_list[i + 1:], start=i + 1):
        distance = euclidean_distance(dict1, dict2)
        if distance > max_distance:
            max_distance = distance
            max_distance_barcodes = (dict1['barcode'], dict2['barcode'])


# Initialize the minimum distance with a large value
min_distance = float('inf')

# Iterate through all pairs of dictionaries and calculate distances
for i, dict1 in enumerate(combined_barcode_list):
    for j, dict2 in enumerate(combined_barcode_list[i + 1:], start=i + 1):
        distance = euclidean_distance(dict1, dict2)
        if distance < min_distance:
            min_distance = distance
            closest_barcodes = (dict1['barcode'], dict2['barcode'])

In [26]:
print(min_distance)
############# Juse to be safe, use min_distance + 1 to find all neighbors

198.0


In [27]:
# Find connected components
G = nx.Graph()

# Add nodes to the graph
for component in combined_barcode_list:
    G.add_node(component['barcode'], x=component['x'], y=component['y'], type=component['type'])

# Define proximity threshold and type matching condition
proximity_threshold = 200

# Iterate through components and add edges based on proximity and type
for i, component1 in enumerate(combined_barcode_list):
    for j, component2 in enumerate(combined_barcode_list[i + 1:], start=i + 1):
        distance = ((component1['x'] - component2['x'])**2 + (component1['y'] - component2['y'])**2)**0.5
        if distance <= proximity_threshold and component1['type'] == component2['type']:
            G.add_edge(component1['barcode'], component2['barcode'])
            # print("this"+component1['id']+component2['id'])

# Print the edges representing the connections
# print("Edges:", list(G.edges()))

connected_components = list(nx.connected_components(G)) # gives a list of sets, each set is a connected clusterfiltered_sets = [s for s in list_of_sets if len(s) > 1]
filtered_connected_components = [s for s in connected_components if len(s) > 1]


# Print the connected components
for idx, component_set in enumerate(filtered_connected_components, start=1):
    component_types = {component['type'] for barcode in component_set for component in combined_barcode_list if component['barcode'] == barcode}
    print(f"Connected Component {idx} (Type: {component_types}): {component_set}")

Connected Component 1 (Type: {'tumor'}): {'GACGCTTGCTTCTAAA-1', 'GAAACTCGTGCGATGC-1', 'AGAGCAGTTATGAGAC-1', 'GTTACAATTGGTGACG-1', 'CATACCCGTACCCAGT-1', 'GCCGCTTGTGAGAAAC-1', 'TGCAGAGTACCGAGCA-1', 'TTGGACATGTGGCTTA-1', 'AATATTGGAGTATTGA-1', 'CATGCCAACTCGCAAA-1', 'ACGTTAGATTTGCCCG-1', 'GTGAGCGTGCTGCACT-1', 'TGGTCTGTTGGGCGTA-1', 'TACAAGTCTCGTGCAT-1', 'GTTTCATATCGTCGCT-1', 'AGCCCATACATGTAAG-1', 'AAGTGACGACCGAATT-1', 'TTGCATGCTGATCACG-1', 'TATGAAGACAGGTGCG-1', 'CTACTCTAGGCCCGGC-1', 'CTAAAGAATGCCTACT-1', 'TTCGTACTCCAGAACG-1', 'GCAGGACTATAGAATA-1', 'GATATGGATTACGCGG-1', 'GAGGCTATCAAAGTCG-1', 'TACGTTTACCGGCAAT-1', 'ACAATCCATTTAAACC-1', 'GATCTTTGCAGGGTAT-1', 'CCCTGCCCAATCCGCT-1', 'CCTGTACTCACGCCCA-1', 'TCTGGGTAGCGCTCAT-1', 'GCATTGTAATTCATAT-1', 'CCGATTCGAGGGACCC-1', 'AACTCAAGTTAATTGC-1', 'CCACTGGTGGCTGGTT-1', 'TCGCGTCCAGAAGGTC-1', 'ATTCCCACATAAACAA-1', 'TTATCGCCTGCGAAGC-1', 'AATAACAACGCTCGGC-1', 'TAGTCTAACAACGAGA-1', 'GACAGGCACACACTAT-1', 'TACTGGACAGCTCGGC-1', 'TCCGATTACATTGCCG-1', 'CCCAATTTCAC

In [28]:
with open('/Users/victoriagao/local_docs/NEST/stored_variables/exp1_C1_filtered_connected_components.pkl', 'wb') as file:
    pickle.dump(filtered_connected_components, file)